In [0]:
# Notebook: Shared_Functions
from pyspark.sql.functions import current_timestamp

def log_pipeline_run(run_id, job_name, layer, status, count, error_msg=""):
    try:
        # We use the table name directly. Databricks handles the storage!
        table_name = "ecomm_data_project.audit.pipeline_logs"
        
        log_data = [(str(run_id), job_name, layer, status, int(count), str(error_msg))]
        columns = ["run_id", "job_name", "layer", "status", "records_processed", "error_message"]
        
        log_df = spark.createDataFrame(log_data, columns).withColumn("timestamp", current_timestamp())
        
        # Append to the Managed Table
        log_df.write.format("delta").mode("append").saveAsTable(table_name)
        print(f"✅ Audit Log written to {table_name}: {status}")
        
    except Exception as e:
        print(f"⚠️ Failed to write Audit Log: {e}")